# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [1]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests
from pathlib import Path

# Import API key
from api_keys import geoapify_key

In [2]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
csv_file = Path("output_data/cities.csv")
city_data_df = pd.read_csv(csv_file)

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,puerto ayora,-0.7393,-90.3518,24.75,95,95,1.34,EC,1712029059
1,1,mari-turek,56.7895,49.6235,4.05,99,100,6.04,RU,1712029060
2,2,pechory,57.8164,27.6119,6.70,95,84,2.01,RU,1712029061
3,3,dunedin,-45.8742,170.5036,14.54,73,14,5.59,NZ,1712028992
4,4,port-aux-francais,-49.3500,70.2167,7.41,91,99,10.56,TF,1712029028


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [13]:
%%capture --no-display

# Configure the map plot
city_map = city_data_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    size = "Humidity",
    frame_width = 800,
    frame_height = 600
)


# Display the map
city_map

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [14]:
city_data_df["Max Temp"].max()

37.72

In [15]:
city_data_df["Max Temp"].min()

-20.09

In [23]:
#First requirement:
ideal_holiday_destinations = city_data_df.loc[(city_data_df["Max Temp"]) <= 32 | (city_data_df["Max Temp"] > 18), :]

In [24]:
ideal_holiday_destinations

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
5,5,bilibino,68.0546,166.4372,-14.67,69,49,0.57,RU,1712029063
6,6,isafjordur,66.0755,-23.1240,-3.03,68,98,10.25,IS,1712029064
7,7,grytviken,-54.2811,-36.5092,1.00,82,23,4.75,GS,1712029065
8,8,iqaluit,63.7506,-68.5145,-0.15,93,100,11.32,CA,1712028784
12,12,st. john's,47.5649,-52.7093,0.93,96,100,3.09,CA,1712029070
...,...,...,...,...,...,...,...,...,...,...
517,517,labytnangi,66.6572,66.4183,-11.94,94,100,2.67,RU,1712029541
527,527,mount pearl,47.5166,-52.7813,0.47,96,100,3.09,CA,1712029549
540,540,labrador city,52.9463,-66.9114,-12.05,92,63,0.00,CA,1712029563
564,564,olyokminsk,60.3736,120.4264,-1.34,93,99,0.98,RU,1712029584


In [25]:
type(ideal_holiday_destinations)

pandas.core.frame.DataFrame

In [26]:
holiday_map = ideal_holiday_destinations.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    size = "Humidity",
    frame_width = 800,
    frame_height = 600
)


# Display the map
holiday_map

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (Humidity)

In [29]:
# Narrow down cities that fit criteria and drop any results with null values
ideal_holiday_destinations = city_data_df.loc[(city_data_df["Max Temp"]) <= 32 | (city_data_df["Max Temp"] > 18), :]

# Drop any rows with null values
ideal_holiday_destinations.count()

# Display sample data
ideal_holiday_destinations

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
5,5,bilibino,68.0546,166.4372,-14.67,69,49,0.57,RU,1712029063
6,6,isafjordur,66.0755,-23.1240,-3.03,68,98,10.25,IS,1712029064
7,7,grytviken,-54.2811,-36.5092,1.00,82,23,4.75,GS,1712029065
8,8,iqaluit,63.7506,-68.5145,-0.15,93,100,11.32,CA,1712028784
12,12,st. john's,47.5649,-52.7093,0.93,96,100,3.09,CA,1712029070
...,...,...,...,...,...,...,...,...,...,...
517,517,labytnangi,66.6572,66.4183,-11.94,94,100,2.67,RU,1712029541
527,527,mount pearl,47.5166,-52.7813,0.47,96,100,3.09,CA,1712029549
540,540,labrador city,52.9463,-66.9114,-12.05,92,63,0.00,CA,1712029563
564,564,olyokminsk,60.3736,120.4264,-1.34,93,99,0.98,RU,1712029584


### Step 3: Create a new DataFrame called `hotel_df`.

In [30]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = ideal_holiday_destinations.copy()

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df["Hotel Name"] = ""

# Display sample data
hotel_df

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
5,5,bilibino,68.0546,166.4372,-14.67,69,49,0.57,RU,1712029063,
6,6,isafjordur,66.0755,-23.1240,-3.03,68,98,10.25,IS,1712029064,
7,7,grytviken,-54.2811,-36.5092,1.00,82,23,4.75,GS,1712029065,
8,8,iqaluit,63.7506,-68.5145,-0.15,93,100,11.32,CA,1712028784,
12,12,st. john's,47.5649,-52.7093,0.93,96,100,3.09,CA,1712029070,
...,...,...,...,...,...,...,...,...,...,...,...
517,517,labytnangi,66.6572,66.4183,-11.94,94,100,2.67,RU,1712029541,
527,527,mount pearl,47.5166,-52.7813,0.47,96,100,3.09,CA,1712029549,
540,540,labrador city,52.9463,-66.9114,-12.05,92,63,0.00,CA,1712029563,
564,564,olyokminsk,60.3736,120.4264,-1.34,93,99,0.98,RU,1712029584,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [ ]:
# Set parameters to search for a hotel
radius = # YOUR CODE HERE
params = {
    # YOUR CODE HERE
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    # YOUR CODE HERE
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = # YOUR CODE HERE
    params["bias"] = # YOUR CODE HERE
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"


    # Make and API request using the params dictionaty
    name_address = # YOUR CODE HERE
    
    # Convert the API response to JSON format
    name_address = # YOUR CODE HERE
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [ ]:
%%capture --no-display

# Configure the map plot
# YOUR CODE HERE

# Display the map
# YOUR CODE HERE